In [23]:
# Import Packages
import sys
from operator import truediv
import gurobipy as gp
from gurobipy import GRB

In [24]:
#########
# MODEL #
#########

m = gp.Model("rotation_scheduling")

########
# SETS #
########

# Defines the (ordered) set of people
people = ["Resident1", "Resident2", "Resident3", "Resident4"]

# Defines the (ordered) set of rotations
rotations = ["Rotation1", "Rotation2", "Rotation3", "Rotation4"]

# Defines the (ordered) set of blocks
blocks = ["1", "2", "3", "4"]

# Defines the set of residents who are around for the entire year
# allYearResidents = {"Resident1": 1, "Resident2": 1, "Resident3": 1, "Resident4": 0}
allYearResidents = ["Resident1", "Resident2", "Resident3"]

# Defines the set of residents who are around for a partial year
# partialYearResidents = {"Resident1": 0, "Resident2": 0, "Resident3": 0, "Resident4": 1}
partialYearResidents = ["Resident4"]

# Defines the rotations that all-year residents must do
# mustDo = {"Rotation1": 1, "Rotation2": 1, "Rotation3": 1, "Rotation4": 0}
mustDo = ["Rotation1","Rotation2","Rotation3" ]

# Defines the rotations that are nonessential
# NOT INCLUDED #

# Defines parameter indicating busy rotations
busyRotations = ["Rotation1", "Rotation2"]

# Defines the rotations during which vacations are allowed
# NOT INCLUDED #

# Defines set of priority assignments
priority = [("Resident2", "Rotation1", "2")]

# Defines set of preference assignments
preference = [("Resident1", "Rotation2", "1")]

# Defines set of impossible assignments
impossibleAssignments = [("Resident3", "Rotation1", "1")]

# Defines set of impossible Rotations in certain blocks 
impossibleRotationInBlock = [("Rotation4","4")]

# Defines vacations or interviews in certain blocks
vacation = [("Resident1", "1"),("Resident1", "4"),("Resident2", "3")]

In [25]:
# Defines parameter for the minimum number of people for each rotation
p_min = {"Rotation1": 1, "Rotation2": 1, "Rotation3": 1, "Rotation4": 0}

# Defines parameter for the minimum number of people for each rotation
p_max = {"Rotation1": 1, "Rotation2": 1, "Rotation3": 2, "Rotation4": 2}

# Defines parameter for the minimum number of times a person must do the specified rotation
# r_min = {[people, rotations]: 0}

# Defines parameter for the maximum number of times a person must do the specified rotation
# r_max = {[people, rotations]: 0}


In [26]:
#############
# DECISION VARIABLES #
#############

# Defines the decision variables (x[p,r,b]=1 if person p assigned to rotation r in block b; x[p,r,b]=0 otherwise)
x = m.addVars(people, rotations, blocks, vtype=GRB.BINARY, name = "x")

# Defines variables for consecutive busy rotations
y = m.addVars(people, busyRotations, busyRotations, blocks, vtype=GRB.BINARY, name = "y")


In [27]:
#############
# OBJECTIVE #
#############

m.setObjective(
    sum(x[(p, r, b)] for (p,r,b) in preference), sense = GRB.MAXIMIZE
)

In [28]:
###############
# CONSTRAINTS #
###############

# Ensures one person cannot be assigned two blocks at once
m.addConstrs((sum(x[(p,r,b)] for r in rotations) == 1  for p in people for b in blocks),name = "personOneAssignmentPerBlock")

{('Resident1', '1'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident1', '2'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident1', '3'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident1', '4'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident2', '1'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident2', '2'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident2', '3'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident2', '4'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident3', '1'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident3', '2'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident3', '3'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident3', '4'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident4', '1'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident4', '2'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident4', '3'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident4', '4'): <gurobi.Constr *Awaiting Model Update*>}

In [29]:
# Ensures sufficient coverage for each rotation
m.addConstrs((p_min[r]  <= sum([x[(p,r,b)] for p in people]) for r in rotations for b in blocks), name = "rotationCoverage_Min" )
m.addConstrs((p_max[r]  >= sum([x[(p,r,b)] for p in people]) for r in rotations for b in blocks), name = "rotationCoverage_Max" )

{('Rotation1', '1'): <gurobi.Constr *Awaiting Model Update*>,
 ('Rotation1', '2'): <gurobi.Constr *Awaiting Model Update*>,
 ('Rotation1', '3'): <gurobi.Constr *Awaiting Model Update*>,
 ('Rotation1', '4'): <gurobi.Constr *Awaiting Model Update*>,
 ('Rotation2', '1'): <gurobi.Constr *Awaiting Model Update*>,
 ('Rotation2', '2'): <gurobi.Constr *Awaiting Model Update*>,
 ('Rotation2', '3'): <gurobi.Constr *Awaiting Model Update*>,
 ('Rotation2', '4'): <gurobi.Constr *Awaiting Model Update*>,
 ('Rotation3', '1'): <gurobi.Constr *Awaiting Model Update*>,
 ('Rotation3', '2'): <gurobi.Constr *Awaiting Model Update*>,
 ('Rotation3', '3'): <gurobi.Constr *Awaiting Model Update*>,
 ('Rotation3', '4'): <gurobi.Constr *Awaiting Model Update*>,
 ('Rotation4', '1'): <gurobi.Constr *Awaiting Model Update*>,
 ('Rotation4', '2'): <gurobi.Constr *Awaiting Model Update*>,
 ('Rotation4', '3'): <gurobi.Constr *Awaiting Model Update*>,
 ('Rotation4', '4'): <gurobi.Constr *Awaiting Model Update*>}

In [30]:
# Ensures that all-year residents must do each must-do rotation
m.addConstrs((sum(x[(p,r,b)] for b in blocks) >= 1  for p in allYearResidents for r in mustDo), name = "AllYear_mustdo")

{('Resident1', 'Rotation1'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident1', 'Rotation2'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident1', 'Rotation3'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident2', 'Rotation1'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident2', 'Rotation2'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident2', 'Rotation3'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident3', 'Rotation1'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident3', 'Rotation2'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident3', 'Rotation3'): <gurobi.Constr *Awaiting Model Update*>}

In [31]:
# Ensures Priority Assignments are fulfilled
m.addConstrs((x[(p,r,b)] == 1 for (p,r,b) in priority), name = "priority")

{('Resident2', 'Rotation1', '2'): <gurobi.Constr *Awaiting Model Update*>}

In [32]:
# Ensures rotations that cannot happen, do not happen
m.addConstrs((x[(p,r,b)] == 0 for (p,r,b) in impossibleAssignments), name = "impossibleAssignment")

{('Resident3', 'Rotation1', '1'): <gurobi.Constr *Awaiting Model Update*>}

In [33]:
# Constraint that establishes certain rotations impossible within certain blocks
m.addConstrs((x[(p,r,b)] == 0 for p in people for (r,b) in impossibleRotationInBlock ), name = "impossibleRotationInBlock")

{('Resident1', 'Rotation4', '4'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident2', 'Rotation4', '4'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident3', 'Rotation4', '4'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident4', 'Rotation4', '4'): <gurobi.Constr *Awaiting Model Update*>}

In [34]:
# Vacations and Interviews constraint that prohibits resident from doing a busy rotation during the vacation or interview period
m.addConstrs((x[(p,r,b)] == 0 for r in busyRotations for (p,b) in vacation), name = "vacation")

{('Rotation1', 'Resident1', '1'): <gurobi.Constr *Awaiting Model Update*>,
 ('Rotation1', 'Resident1', '4'): <gurobi.Constr *Awaiting Model Update*>,
 ('Rotation1', 'Resident2', '3'): <gurobi.Constr *Awaiting Model Update*>,
 ('Rotation2', 'Resident1', '1'): <gurobi.Constr *Awaiting Model Update*>,
 ('Rotation2', 'Resident1', '4'): <gurobi.Constr *Awaiting Model Update*>,
 ('Rotation2', 'Resident2', '3'): <gurobi.Constr *Awaiting Model Update*>}

In [35]:
# Defines y

m.addConstrs((y[(p,r1,r2,b1)] <= (2 - x[(p,r1,b1)] - x[(p,r2,b2)]) for p in people for r1 in busyRotations for r2 in busyRotations for b1,b2 in zip(blocks, blocks[1:])), name = "consecutiveBusyRotation")


{('Resident1',
  'Rotation1',
  'Rotation1',
  '1',
  '2'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident1',
  'Rotation1',
  'Rotation1',
  '2',
  '3'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident1',
  'Rotation1',
  'Rotation1',
  '3',
  '4'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident1',
  'Rotation1',
  'Rotation2',
  '1',
  '2'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident1',
  'Rotation1',
  'Rotation2',
  '2',
  '3'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident1',
  'Rotation1',
  'Rotation2',
  '3',
  '4'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident1',
  'Rotation2',
  'Rotation1',
  '1',
  '2'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident1',
  'Rotation2',
  'Rotation1',
  '2',
  '3'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident1',
  'Rotation2',
  'Rotation1',
  '3',
  '4'): <gurobi.Constr *Awaiting Model Update*>,
 ('Resident1',
  'Rotation2',
  'Rotation2',
  '1',
  '2'): <gurobi.Constr *Awaiting Model 

In [36]:
m.optimize()
a = m.getObjective()
print(a.getValue())

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 117 rows, 128 columns and 384 nonzeros
Model fingerprint: 0x5bf1260b
Variable types: 0 continuous, 128 integer (128 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 117 rows and 128 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: -0 
No other solutions better than -0

Optimal solution found (tolerance 1.00e-04)
Best objective -0.000000000000e+00, best bound -0.000000000000e+00, gap 0.0000%
0.0


In [37]:
for v in m.getVars():
    if v.x == 1: # When assign the resident to this rotation in the block 
        # print('%s %g' % (v.varName,v.x))
        print(v.varName)

x[Resident1,Rotation1,3]
x[Resident1,Rotation2,2]
x[Resident1,Rotation3,1]
x[Resident1,Rotation3,4]
x[Resident2,Rotation1,2]
x[Resident2,Rotation2,4]
x[Resident2,Rotation3,3]
x[Resident2,Rotation4,1]
x[Resident3,Rotation1,4]
x[Resident3,Rotation2,1]
x[Resident3,Rotation3,2]
x[Resident3,Rotation4,3]
x[Resident4,Rotation1,1]
x[Resident4,Rotation2,3]
x[Resident4,Rotation3,2]
x[Resident4,Rotation3,4]
